***Author***: [Manh Cuong](https://facebook.com/cuongpigerr)<br>
***Date***: 13/04/2020 - 02:32PM

In [17]:
from functools import reduce
import operator
import itertools

In [18]:
def readData(file):
    S = []
    n = 0
    
    f = open(file, 'r')
    seq = f.readline().strip()
    
    while seq != '':
        n += 1
        pats = seq.split(',')
        line = []
        
        for p in pats:
            pat = p.split(' ')
            line.append(pat)
            
        S.append(line)
        seq = f.readline().strip()
        
    return (S, n)

In [19]:
def initPass(S):
    res = dict()
    
    for seq in S:
        items = reduce(lambda a, b: set(a).union(set(b)), seq)
        
        for item in items:
            res[item] = res.get(item, 0) + 1
          
    res = dict(sorted(res.items(), key=operator.itemgetter(0)))
    return res

In [20]:
def genF1(supItems, n, minsup):
    F1 = []
    
    for key, val in supItems.items():
        if val/n*100.0 >= minsup:
            F1.append([])
            F1[-1].append(tuple([key]))
            
    return F1

In [21]:
def compare2seq(seq1, seq2):
    s1 = list(map(lambda x: list(x), seq1))
    s2 = list(map(lambda x: list(x), seq2))
    tmp1 = list(map(lambda x: list(x), seq1))
    tmp2 = list(map(lambda x: list(x), seq2))

    res = []
    
    del tmp1[0][0]
    del tmp2[-1][-1]
    
    if not tmp1[0]:
        tmp1.pop(0)
        
    if not tmp2[-1]:
        tmp2.pop()
        
    if tmp1 == tmp2:
        tmp = list(map(lambda x: list(x), seq1))

        if len(s2[-1]) == 1:
            s1.append(s2[-1])
        else:
            s1[-1].append(s2[-1][-1])

        res = s1
    
    return res

In [22]:
def findSubsets(s, n): 
    return list(map(list, itertools.combinations(s, n)))

In [23]:
def findSubseqs(c):
    subseqs = []
    
    for i in range(len(c)):
        subsets = findSubsets(c[i], len(c[i]) - 1)
        head = c[:i]
        tail = c[i + 1:]
        
        for sub in subsets:
            tmp = head + [sub] + tail
            tmp = [x for x in tmp if x]
            tmp = [tuple(x) for x in tmp]
            subseqs.append(tmp)
            
    return subseqs

In [24]:
def candidateK2(F1):
    C2 = list()
    
    # [tuple([1, 2]), tuple([4])]
    for i in range(len(F1) - 1):
        for j in range(i + 1, len(F1)):
            s1, s2 = F1[i][0][0], F1[j][0][0]
            
            c1 = [tuple([s1, s2]),]
            c2 = [tuple([s1]), tuple([s2])]
            c3 = [tuple([s2]), tuple([s1])]
            
            C2.append(c1)
            C2.append(c2)
            C2.append(c3)
            
    return C2

In [25]:
def candidateGenSPM(Fkminus1, k):
    Ck = list()
    
    for i in range(len(Fkminus1) - 1):
        for j in range(i + 1, len(Fkminus1)):
            s1, s2 = Fkminus1[i], Fkminus1[j]
            c1 = compare2seq(s1, s2)
            c1_subs = findSubseqs(c1)
            
            c2 = compare2seq(s2, s1)
            c2_subs = findSubseqs(c2)
            
            flag = True
            for sub in c1_subs:
                if sub not in Fkminus1:
                    flag = False
                    break
                
            if flag:
                Ck.append([tuple(sorted(x)) for x in c1])
                
            flag = True
            for sub in c2_subs:
                if sub not in Fkminus1:
                    flag = False
                    break
                
            if flag:
                Ck.append([tuple(sorted(x)) for x in c2])
                    
    res = []
    for c in Ck:
        if c not in res:
            res.append(c)
    
    return res

In [26]:
def genC1(items):
    C1 = []
    
    for item in items:
        C1.append([])
        C1[-1].append(tuple([item]))
        
    return C1

In [27]:
def customiseS(S):
    s = []

    for i in range(len(S)):
        s.append([])
        s[-1].append(list(map(lambda x: tuple(x), S[i])))
        
    return s

In [28]:
def isSubset(s, s_sub):
    return set(s_sub).issubset(set(s))

In [29]:
def isSubseq(S, C):
    if len(S) <  len(C):
        return False

    i, j = 0, 0
    
    while i < len(S) and j < len(C):
        if isSubset(S[i], C[j]):
            i += 1
            j += 1
        else:
            i += 1

    return j == len(C)

In [30]:
def printResult(C, F):
    print('Bảng C')
    for i in range(1, len(C)):
        print('C{0}: {1}'.format(i, C[i]))
        
    print('\nBảng F')
    for i in range(1, len(F)):
        print('F{0}: {1}'.format(i, F[i]))

In [31]:
def gsp(file, minsup):
    C, F = [-1], [-1]
    k = 2
    
    S, n = readData(file)
    supItems = initPass(S)
    
    C1 = genC1(list(supItems.keys()))
    F1 = genF1(supItems, n, minsup)
    
    C.append(C1)
    F.append(F1)
    # S = customiseS(S)
    
    while F[k - 1]:
        Ck = None
        
        if k == 2:
            Ck = candidateK2(F[1])
        else:
            Ck = candidateGenSPM(F[k - 1], k)
            
        cCount = {}
        for s in S:
            for c in Ck:
                if isSubseq(s, c) and c != []:
                    cCount[tuple(c)] = cCount.get(tuple(c), 0) + 1
        
        Ck = [x for x in Ck if x]
        Fk = []
        for c in Ck:
            if tuple(c) in cCount:
                if cCount[tuple(c)]/n*100.0 >= minsup:
                    Fk.append(c)
            
        C.append(Ck)
        F.append(Fk)
        k += 1
        
    printResult(C, F)

In [32]:
gsp('data1.txt', 50.0)

Bảng C
C1: [[('a',)], [('b',)], [('c',)], [('d',)], [('e',)], [('f',)], [('g',)]]
C2: [[('a', 'b')], [('a',), ('b',)], [('b',), ('a',)], [('a', 'c')], [('a',), ('c',)], [('c',), ('a',)], [('a', 'd')], [('a',), ('d',)], [('d',), ('a',)], [('a', 'e')], [('a',), ('e',)], [('e',), ('a',)], [('a', 'f')], [('a',), ('f',)], [('f',), ('a',)], [('b', 'c')], [('b',), ('c',)], [('c',), ('b',)], [('b', 'd')], [('b',), ('d',)], [('d',), ('b',)], [('b', 'e')], [('b',), ('e',)], [('e',), ('b',)], [('b', 'f')], [('b',), ('f',)], [('f',), ('b',)], [('c', 'd')], [('c',), ('d',)], [('d',), ('c',)], [('c', 'e')], [('c',), ('e',)], [('e',), ('c',)], [('c', 'f')], [('c',), ('f',)], [('f',), ('c',)], [('d', 'e')], [('d',), ('e',)], [('e',), ('d',)], [('d', 'f')], [('d',), ('f',)], [('f',), ('d',)], [('e', 'f')], [('e',), ('f',)], [('f',), ('e',)]]
C3: [[('c',), ('a', 'b')], [('e',), ('a', 'b')], [('a', 'b'), ('c',)], [('a', 'b'), ('d',)], [('a', 'b'), ('f',)], [('c',), ('a',), ('b',)], [('e',), ('a',), ('b',